<!--- mission_number=5 file_list=["nba_2013.csv"] mode="singlescreen" --->

# K-nearest neighbors introduction
## Use the knn algorithm to predict NBA statistics.
## Vik Paruchuri

<!--- type="code" --->

# A look at the data

Before we get started with the KNN algorithm, let's take a look at our data.  Each row in the data contains information on how a player performed in the 2013-2014 NBA season.

Here are some selected columns:

* `player` -- name of the player
* `pos` -- the position of the player
* `g` -- number of games the player was in
* `gs` -- number of games the player started
* `pts` -- total points the player scored

See [this site](http://www.databasebasketball.com/about/aboutstats.htm) for an explanation of the rest of them.

In [14]:
import pandas
with open("nba_2013.csv", 'r') as csvfile:
    nba = pandas.read_csv(csvfile)

# The names of the columns in the data.
print(nba.columns.values)

['player' 'pos' 'age' 'bref_team_id' 'g' 'gs' 'mp' 'fg' 'fga' 'fg.' 'x3p'
 'x3pa' 'x3p.' 'x2p' 'x2pa' 'x2p.' 'efg.' 'ft' 'fta' 'ft.' 'orb' 'drb'
 'trb' 'ast' 'stl' 'blk' 'tov' 'pf' 'pts' 'season' 'season_end']


<!--- type="text" --->

# KNN overview

The k-nearest neighbors is based around the simple idea of predicting unknown values by matching them with the most similar known values.

Let's say that we have 3 different types of cars:

```
car,horsepower,racing_stripes,is_fast
Honda Accord,180,False,False
Yugo,500,True,True
Delorean DMC-12,200,True,True
```

Let's say that we now have another car:

```
Chevrolet Camaro,400,True,Unknown
```

We don't know whether or not this car is fast.  In order to predict if it is, we find the most similar known car.  In this case, we would compare the `horsepower` and `racing_stripes` values to find the most similar car, which is the `Yugo`.  Since the Yugo is fast, we would predict that the Camaro is also fast.  This is an example of 1-nearest neighbors -- we only looked at the most similar car.

If we performed a 2-nearest neighbors, we would end up with 2 `True` values (for the Delorean and the Yugo), which would average out to `True`.

If we did 3-nearest neighbors, we would end up with 2 `True` values and a `False` value, which would average out to `True`.

<!--- type="code" --->

# Euclidean distance

Before we can predict using KNN, we need to find some way to figure out which data rows are "closest" to the row we're trying to predict on.

A simple way to do this is to use Euclidean distance.  The formula is $$\sqrt{(q_1-p_1)^2 + (q_2-p_2)^2 + \cdots + (q_n-p_n)^2}$$

Let's say we have these two rows (True/False has been converted to 1/0), and we want to find the distance between them:

```
Honda Accord,180,0
Chevrolet Camaro,400,1
```

We would first only select the numeric columns.  Then the distance becomes $$\sqrt{(180-400)^2 + (0-1)^2}$$, which is about equal to `220`.

## Instructions

Make a function for calculating the euclidean distance between two pandas series.  Use the function to find the euclidean distance between `selected_player` and each row in `nba`.  Use the `.apply(func, axis=1)` method on dataframes to apply function `func` to each row.  The function should take `row` as its first argument.  Only use the columns in `distance_columns` to compute the distance.  [Here's](http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DataFrame.apply.html) more on the method.

Assign the resulting pandas series to `lebron_distance`.

## Hint

`selected_player` and the row passed into the euclidean distance function will both be pandas series, and you can get a column value by indexing the series by name.  `row["fg"]` will get you the number of field goals if `row` is a series.

In [63]:
## Initial

# Fill in any NA values without the student knowing.
nba.fillna(0, inplace=True)

## Display

selected_player = nba[nba["player"] == "LeBron James"].iloc[0]
distance_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

## Answer

import math
def euclidean_distance(row):
    inner_value = 0
    for k in distance_columns:
        inner_value += (row[k] - selected_player[k]) ** 2
    return math.sqrt(inner_value)

lebron_distance = nba.apply(euclidean_distance, axis=1)

## Check vars

["lebron_distance"]

['lebron_distance']

<!--- type="code" --->

# Normalizing columns

You may have noticed that `horsepower` in the last example had a much larger impact on the final distance than `racing_stripes` did.  This is because `horsepower` values are much larger in absolute terms, and thus dwarf the impact of `racing_stripes` values in the euclidean distance calculations.

This can be bad, because a variable having larger values doesn't necessarily make it better at predicting what rows are similar.

A simple way to deal with this is to normalize all the columns to have a mean of 0, and a standard deviation of 1.  This will ensure that no single column has a dominant impact on the euclidean distance calculations.

To set the mean to 0, we have to find the mean of a column, then subtract the mean from every value in the column.  To set the standard deviation to 1, we divide every value in the column by the standard deviation.  The formula is $$x=\frac{x-\mu}{\sigma}$$.

## Instructions

Normalize the columns in `nba_numeric`.  Using `.mean()` on a dataframe will return the mean of each column.  Using `.std()` will return the standard deviation of each column.

## Hint

Subtracting `nba_numeric.mean()` from `nba_numeric` will subtract the mean from each column.  You can follow the same process to divide by standard deviation.

In [23]:
## Display

nba_numeric = nba[distance_columns]

## Answer

nba_normalized = (nba_numeric - nba_numeric.mean()) / nba_numeric.std()

## Check vars

["nba_normalized"]

['nba_normalized']

<!--- type="code" --->

# Finding the nearest neighbor

We now know enough to find the nearest neighbor of a given row.  We can use the `distance.euclidean` function from `scipy.spatial`, a much faster way to calculate euclidean distance.

## Instructions

Find the player most similar to LeBron James by our distance metric.  You can do this by finding the second lowest value in the `euclidean_distances` series (the lowest value will correspond to lebron, as he is most similar to himself), and then cross-referencing the nba dataframe with the same index.

Assign the name of the player to `most_similar_to_lebron`.

## Hint

You can make a new dataframe using `distance_frame = pandas.DataFrame(data={"dist": euclidean_distances, "idx": euclidean_distances.index})`.  Then you can sort the dataframe on a column using `.sort("column_name")`.  Then it's a matter of picking the second row, finding the `idx` column value, and using that value to get the correct row in the `nba` dataframe.

In [46]:
## Display

from scipy.spatial import distance

# Fill in NA values in nba_normalized
nba_normalized.fillna(0, inplace=True)

# Find the normalized vector for lebron james.
lebron_normalized = nba_normalized[nba["player"] == "LeBron James"]

# Find the distance between lebron james and everyone else.
euclidean_distances = nba_normalized.apply(lambda row: distance.euclidean(row, lebron_normalized), axis=1)

## Answer

distance_frame = pandas.DataFrame(data={"dist": euclidean_distances, "idx": euclidean_distances.index})
distance_frame.sort("dist", inplace=True)
second_smallest = distance_frame.iloc[1]["idx"]
most_similar_to_lebron = nba.loc[int(second_smallest)]["player"]

## Check vars

["most_similar_to_lebron"]

['most_similar_to_lebron']

<!--- type="code" --->

# Generating training and testing sets

Now that we know how to find the nearest neighbors, we can make predictions on a test set.

First, we have to generate test and train sets.  In order to do this, we'll use random sampling.  We'll randomly shuffle the index of the `nba` dataframe, and then pick rows using the randomly shuffled values.

If we didn't do this, we'd end up predicting and training on the same data set, which would overfit.  We could do cross validation also, which would be slightly better, but slightly more complex.

In [64]:
import random
from numpy.random import permutation

# Randomly shuffle the index of nba.
random_indices = permutation(nba.index)
# Set a cutoff for how many items we want in the test set (in this case 1/3 of the items)
test_cutoff = math.floor(len(nba)/3)
# Generate the test set by taking the first 1/3 of the randomly shuffled indices.
test = nba.loc[random_indices[1:test_cutoff]]
# Generate the train set with the rest of the data.
train = nba.loc[random_indices[test_cutoff:]]

<!--- type="code" --->

# Using sklearn

Instead of having to do it all ourselves, we can use the k-nearest neighbors implementation in scikit-learn.  [Here's](http://scikit-learn.org/stable/modules/neighbors.html) the documentation.  There's a regressor and a classifier available, but we'll be using the regressor, as we have continuous values to predict on.

Sklearn performs the normalization and distance finding automatically, and lets us specify how many neighbors we want to look at.

In [73]:
# The columns that we will be making predictions with.
x_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']
# The column that we want to predict.
y_column = ["pts"]

from sklearn.neighbors import KNeighborsRegressor
# Create the knn model.
knn = KNeighborsRegressor(n_neighbors=5)
# Fit the model on the training data.
knn.fit(train[x_columns], train[y_column])
# Make predictions on the test set using the fit model.
predictions = knn.predict(test[x_columns])

<!--- type="code" --->

# Computing error

Now that we know our predictions, we can compute the error involved.  We can compute [mean squared error](http://en.wikipedia.org/wiki/Mean_squared_error).  The formula is $$\frac{1}{n}\sum_{i=1}^{n}(\hat{y_{i}} - y_{i})^{2}$$.

## Instructions

Compute the mean squared error between `actual` and `predictions`.  Assign the result to `mse`.

## Hint

Follow the formula -- you can subtract all the values in predictions and actual using `predictions - actual`.

In [72]:
## Display

actual = test[y_column]

## Answer

mse = (((predictions - actual) ** 2).sum()) / len(predictions)

## Check vars

["mse"]

['mse']